In [3]:
import bs4 as bs
import pickle
import requests
import datetime as dt
import os
import pandas as pd
import pandas_datareader.data as web

# Pretend use browser
headers = {'User-Agent': 'Mozilla/75.0 Chrome/80.0.3987.163 '}

In [5]:
#  stock symbol
def save_nq100_symbols():
    
    # crawler websites
    resp = requests.get('https://en.wikipedia.org/wiki/NASDAQ-100',headers = headers)

    # source is text, parser is lxml
    soup = bs.BeautifulSoup(resp.text, 'lxml')
    
    # table
    table = soup.find_all("table",{"id" : "constituents"})[0]
    
    
    symbols = []
    
    #tr is row of table
    for row in table.findAll('tr')[1:]:
        
        # soup object get text)
        symbol = row.findAll('td')[0].get_text(strip=True)

        symbols.append(symbol)
        
    #  save result to pickle
    with open("NQ100.pickle","wb") as file:
        pickle.dump(symbols, file)
    
    return symbols


In [6]:
save_nq100_symbols()

['Activision Blizzard',
 'Adobe Inc.',
 'Advanced Micro Devices',
 'Alexion Pharmaceuticals',
 'Align Technology, Inc.',
 'Alphabet Inc.(Class A)',
 'Alphabet Inc.(Class C)',
 'Amazon.com',
 'American Airlines Group',
 'Amgen Inc.',
 'Analog Devices',
 'ANSYS',
 'Apple Inc.',
 'Applied Materials, Inc.',
 'ASML Holding',
 'Autodesk, Inc.',
 'Automatic Data Processing, Inc.',
 'Baidu.com, Inc.',
 'Biogen, Inc',
 'BioMarin Pharmaceutical, Inc.',
 'Booking Holdings',
 'Broadcom Inc.',
 'Cadence Design Systems',
 'CDW',
 'Cerner Corporation',
 'Charter Communications, Inc.',
 'Check Point Software Technologies Ltd.',
 'Cintas Corporation',
 'Cisco Systems',
 'Citrix Systems',
 'Cognizant Technology Solutions Corporation',
 'Comcast Corporation',
 'Copart',
 'CoStar Group',
 'Costco Wholesale Corporation',
 'CSX Corporation',
 'Dollar Tree, Inc.',
 'eBay Inc.',
 'Electronic Arts',
 'Exelon Corporation',
 'Expedia Group',
 'Facebook, Inc.',
 'Fastenal Company',
 'Fiserv, Inc.',
 'Fox Corporat

In [7]:
# save price from spider
def get_data_from_datareader(beg_y, beg_m, beg_d, end_y, end_m, end_d, reload_symbols=False):

    # if reload, execute save_nq100_symbols()
    if reload_symbols:
        symbols = save_nq100_symbols()
    # if no reload, read NQ100.pickle     
    else:
        with open("NQ100.pickle","rb") as file:
            symbols = pickle.load(file)
    
    # if no folder exist, mkdir
    if not os.path.exists('NQ100'):
        os.makedirs('NQ100')
        
    beg = dt.datetime(beg_y, beg_m, beg_d)
    end = dt.datetime(end_y, end_m, end_d)
#     print(beg,end)

    #for loop price from 'stooq'
    for symbol in symbols:
        #try: check symbol is downloaed
        try:
            if not os.path.exists('NQ100/{}.csv'.format(symbol)):
                df = web.DataReader(symbol, 'stooq', beg, end)
                df.to_csv('NQ100/{}.csv'.format(symbol))
            else:
                print('already have {}'.format(symbol))
        # except
        except:
            print(symbol + "can not download")
    print("crawler done")

In [8]:
get_data_from_datareader(2020,1,1,2020,4,17)

crawler done
